<p style="text-align: center;"><span style="color:#262626"><span style="font-size:42px"><span style="font-family:lato,sans-serif">
**SCF Workshop**
</span></span></span></p>

<p style="text-align: left;"><span style="color:#262626"><span style="font-size:18px"><span style="font-family:lato,sans-serif">
**By:** Lucas Aebersold 
</span></span></span></p>
<p style="text-align: left;"><span style="color:#262626"><span style="font-size:18px"><span style="font-family:lato,sans-serif">
Summer 2018
</span></span></span></p>
<p style="text-align: justify;"><span style="color:#262626"><span style="font-size:16px"><span style="font-family:lato,sans-serif">
In this workshop we will be creating our very own Hartree Fock code! Now, when someone asks if you remember making your first HF code, you won't actually be lying!! Let's begin!
</span></span></span></p>
***


This code and discussion is based from Chapter 3. and the code is inspired from the Fortran IV code given in Appendix B. of the sacred text on Quantum Chemistry: *Modern Quantum Chemistry*, Introduction to Advanced Electronic Structure Theory. by Szabo and Ostlund[1], Which if you don't have, go and get it! Order it of off  [Amazon](https://www.amazon.com/Modern-Quantum-Chemistry-Introduction-Electronic/dp/0486691861/ref=sr_1_5?ie=UTF8&qid=1526232742&sr=8-5&keywords=quantum+chemistry&dpID=41mbqZjAm6L&preST=_SY291_BO1,204,203,200_QL40_&dpSrc=srch), it's like 20\$ new, and like 5\$ used.
I have pdf copies too, which most of you have access to, but if you don't have physical copy, it will be hard for people to take you serious in the field of quantum chemistry. 

This workshop tutorial is also based on some of the HF tutorials created by the Psi4 developers, in their psi4numpy python package. I suggest you check it out, and later in the tutorials we will be using this package. 
[Psi4numpy](https://github.com/psi4/psi4numpy)
### References
1. A. Szabo and N. S. Ostlund, *Modern Quantum Chemistry*, Introduction to Advanced Electronic Structure Theory. Courier Corporation, 1996.

Let's import the libraries we need

I do the `*` import for numpy, which imports everything and can be called directly. In most cases this isn't a good programming practice, but in my (our) case we are never not using numpy. I also absolutely hate code that has all those freaking np.cos, np.pi, np.exp, it looks very unnatural and unnecessarily convoluted, and doesn't match up to other codes very well, e.g. fortran, C, etc. 
Will there be compatibility issues with other libraries? Probably some obscure ones, definitely the `random` library, but why are you using the random library when you can just use the one in numpy? Generally, only with `SymPy` will I ever use the `import numpy as np`, else I import it all.

In [1]:
from numpy import *
from scipy.linalg import eigh, eig, inv
from scipy.special import erf
from scipy.constants import physical_constants
au2eV = physical_constants['Hartree energy in eV'][0]
bohr2ang = physical_constants['Bohr radius'][0]*1e10

Ok, these are just printing functions I've defined for later. You don't have to worry about what they're doing
for the sake of understanding this tutorial, they are just here to print out the results in a clean and readable fashion

In [2]:
bohr2ang = 0.529177
# prints nice form of matrix 
def print_matrix(mat, header, print_file=None):

    cols = mat.shape[0]
    rows = mat.shape[1]

    str_mat ='{:^{width}}\n'.format(header, width=rows*10)
    for i in range(cols):
        str_mat += '{}\n'.format(' '.join('{:> .6f}'.format(x) for x in mat[i, :]))
    print(str_mat, file=print_file)
    
def print_matrix_center(mat, header, print_file=None):

    cols = mat.shape[0]
    rows = mat.shape[1]

    str_mat ='{:^{width}}\n'.format(header, width=rows*20+9)
    for i in range(cols):
        str_mat += '{:^{width}}\n'.format(' '.join('{:> .6f}'.format(x) for x in mat[i, :]), width=rows*20+9)
    print(str_mat, file=print_file)
    
def print_matrices(mat, mat2, header, print_file=None):

    cols = mat.shape[0]
    rows = mat.shape[1]

    str_mat ='{:^{width}}'.format(header[0], width=rows*9)
    str_mat += '{:^{width}}'.format(' ', width=9)
    str_mat +='{:^{width}}\n'.format(header[1], width=rows*10)
    for i in range(cols):
        str_mat += '{}'.format(' '.join('{:> 8.6f}'.format(x) for x in mat[i, :]))
        str_mat += '{:^{width}}'.format(' ', width=9)
        str_mat += '{}\n'.format(' '.join('{:> 8.6f}'.format(x) for x in mat2[i, :]))
    print(str_mat, file=print_file)
    
# prints vector occupation and orbital energies
def print_diag(mat, print_file=None):
    diag_vals = mat.diagonal()
    nel = 2
    str_mat ='{:^80}\n'.format('{:^{width}}         Energy'.format('MO', width=10))
    for i in range(len(diag_vals)):
        str_mat += '{:^80}\n'.format('Vector {}  Occ={}  {: .6f}'.format(i+1, nel, diag_vals[i]))
        nel -= 2 
    print(str_mat, file=print_file)
    
# prints all matrices during scf cycle
def verbose_iter_print(print_file):
    print('{:*^80}\n'.format(" Iteration {:2} ".format(scf_iter + 1)), file=print_file)
    print_matrices(F, Fprime, ['F matrix', "F' matrix"], print_file=print_file)
    print_matrices(C, Cprime, ['C matrix', "C' matrix"], print_file=print_file)
    print_matrix_center(P, 'P matrix', print_file=print_file)    
    print('{:<20} = {: .11f}'.format('Electronic Energy', E), file=print_file)
    print('{:<20} = {: .11f}'.format('Total Energy', ET), file=print_file)
    print('Delta (Convergence of the density matrix) = {:.6e}'.format(delta), file=print_file)
    
def print_error_quit():
    print('FATAL ERROR\nGROSSE FEHLER\nCALCULATION DID NOT CONVERGE!', file=f)
    print_diag(Eocc, print_file=f)
    print('{:<18} = {: .11f}'.format('Electronic Energy', E), file=f)
    print('{:<18} = {: .11f}'.format('Total Energy', ET), file=f)
    
def print_convergence_success(Eocc, orthog='canonical'):
    print('{:^80}\n{:^80}'.format('CALCULATION CONVERGED!!!!', 'OMG!!!!!'), file=f)
    print('{:_^80}'.format(' Final Values (atomic units) '), file=f)
    if orthog == 'canonical':
        print_diag(Eocc, print_file=f)
    elif orthog == 'symmetric':
        Eocc = Eocc*eye(2)
        print_diag(Eocc, print_file=f)
        
    print('{:<18} = {: .11f}'.format('Electronic Energy', E), file=f)
    print('{:<18} = {: .11f}'.format('Total Energy', ET), file=f)

I've also predefined these functions as they are usually intrinsic routines available in most programming languages. 
Not all of them are used in this program, but it's good to know a little bit about what tasks they are actually doing. 

The `F0` will should look familiar from my presentation and notes. 

In [3]:
# this calculates the F0 function, but for only s-type orbitals (l = 0) hence F0 
# two options, see next function for description
def F0(arg, oldskool=False):
    if arg < 1.0e-6:
        F0 = 1.0 - arg/3.0
    else:
        if oldskool:
            F0 = sqrt(pi/arg)*derf(sqrt(arg))/2.0
        else:
            F0 = sqrt(pi/arg)*erf(sqrt(arg))/2.0
    return F0

A routine to calculate the `F0` function using either scipy's built in erf routine or the derf routine, defined in S&O, using this function you'll get their results exactly.
Otherwise the `F0` function calls scipy's erf routine which gives a slightly lower energy value
due to the better calculation scheme used.

Note: why does the Szabo and Ostlund routine have a d in front? It stands for 'double precision', a common thing in early fortran programs to distinguish the function from a single precision one. Precision indicating how many decimal places are used in the calculation, look up floating point operations for more info. 

In [4]:
def derf(arg):
    P = 0.3275911
    A = [0.254829592, -0.284496736, 1.421413741, -1.453152027, 1.061405429]
    T = 1.0/(1.0 + P*arg)
    TN = T
    poly = A[0]*TN
    for i in range(1, 5):
        TN *= T
        poly += A[i]*TN
    derf = 1.0 - poly*exp(-arg**2)
    return derf

Diagonalization routine, specific for two-by-two matrices
An alternative is to use numpy's eigh function which is a wrapper to the DYSEV BLAS routine
This is to show the inner workings of the function so you have a better idea of what everything is doing

In [ ]:
def diag(F):
    
    if (abs(F[0,0] - F[1,1]) > 1.0e-20):
        theta = 0.5*arctan(2*F[0,1]/(F[0,0] - F[1,1]))
    else:
        theta = pi/4
        
    C = array([
        [cos(theta),  sin(theta)],
        [sin(theta), -cos(theta)]
    ])
    
    E = zeros((2,2))
    E[0,0] = F[0,0]*cos(theta)**2 + F[1,1]*sin(theta)**2 + F[0,1]*sin(2*theta)
    E[1,1] = F[1,1]*cos(theta)**2 + F[0,0]*sin(theta)**2 - F[0,1]*sin(2*theta)

    if E[1,1] < E[0,0]:
        
        E[1,1], E[0,0] = E[0,0], E[1,1]
        C[0,1], C[0,0] = C[0,0], C[0,1]
        C[1,1], C[1,0] = C[1,0], C[1,1]

    return  E, C

A showcase of the loop structure of matrix-matrix multiplication (i.e. an inner product), 
numpy/scipy's dot will be used in its place, as these are wrappers for the BLAS (Basic Linear Algebra Subroutines) routines and can handle general vector-vector (Level 1) vector-matrix (level 2) and matrix-matrix (level 3) operations

In [ ]:
def mult(A, B, M):
    C = zeros((M, M))
    for i in range(0, M):
        for j in range(0, M):
            for k in range(0, M):
                C[i,j] += A[i,k]*B[k,j]
    return C

Now we begin 

We first define some functions for the overlap $S$, the kinetic energy $T$ and the potential energy $V$. Then also the two-electron repulsion integral $V_{ee}$ and a way to form our $G$ matrix.

In [ ]:
# the overlap  integral for unnormalized primitives
def S(A, B, RAB2):
#     S = ...
    S = (pi/(A + B))**1.5*exp(-A*B*RAB2/(A + B))
    return S

# the kinetic energy integrals for unnormalized primitives
def T(A, B, RAB2):
#     T = ...
    return T

# the potential energy nuclear attraction integrals, we use F0 here and this will only hold for s-type functions
def V(A, B, RAB2, RCP2, ZC):
#     V = ...

    return V

# the two electron integral for unnormalized primitives 
# A, B, C, D are the exponents alpha, beta, etc. 
# RAB2 = squared distance bewteen center A and center B, etc. 
# Again, F0 is used so DON'T USE IT FOR P-orbitals!!
def twoe(A, B, C, D, RAB2, RCD2, RPQ2):
#     Nfac = ...
#     twoe = ...'
    return twoe

# form the G-matrix from the electron-electron repulsion and density matrices
def formG(P):
    g = zeros((nel,nel))
    # four index loop
    
    return g

### Setting up 
Ok, let's set up the parameters for the calculations. We need to tell it 
* How many electrons to use, 
* The positions of the atoms, 
* What kind of atoms they are 
* And lastly the basis set coefficients and exponents. 

### The parameters 

In [ ]:
# STO-NG calc for N = 1, 2 or 3. we will use the STO-3G basis
N = 3  
n = N - 1        # n is used to index the coeff/expon properly, as python arrays/lists follow the 
                 # screwy C-convention of starting at zero and going up to the half open interval
                 # i.e. [0, 3) -> [0, 1, 2]
        
nel = 2          # number of electrons -> two
R = 1.4632       # this is the He-H bond distance, in Bohr

ZA = 2.0         # Atomic number for He
ZB = 1.0         # Atomic number for H
R2 = R*R         # the radius squared

# Define nuclear repulsion energy -> it will not change throughout scf cycle 
E_nuc = ...

### The basis set
We will set up arrays containing our basis set coefficients and basis set exponents for the STO-NG parameters. 

Note, the $\zeta_1$ value for He used is not the standard STO-NG scaling factor as would appear in the EMSL basis entry for the STO-3G set. We follow the discussion of S&O, and you can read a precise explanation of the origin of this other $\zeta$ value on pg 170 of S&O 

We will find this zeta value is slightly better than the standard, we will do the calculation for S&O exponent first  and later use the standard value for comparison in Psi4. 

Basis sets of the STO-3G taken from EMSL. 
```
#BASIS SET: (3s) -> [1s]
H    S
      3.42525091             0.15432897       
      0.62391373             0.53532814       
      0.16885540             0.44463454       
#BASIS SET: (3s) -> [1s]
He    S
      6.36242139             0.15432897       
      1.15892300             0.53532814       
      0.31364979             0.44463454 
```

Now, I have some illustrative examples in this Jupyter notebook [Basis sets](Minimal_Basis_Sets.ipynb)
You will need widgets properly configured to enjoy the full experience, but if you are technilogically challenged 
and cannot configure ipython widgets properly, than you don't deserve the full experience. 

In [ ]:
zeta1 = 2.0925   # Slater orbital exponent zeta for He
# zeta1 = 1.69 # Standard orbital exponent zeta for He 
zeta2 = 1.24     # Slater orbital exponent zeta for H

# these are the coefficients and exponents of the STO-NG bases
# the exponents turn into the STO-NG basis sets for the respective
# atom when multiplied by the appropriate zeta
coeff_sto = array([
    [1.000000, 0.000000, 0.000000], # STO-1G coeff
    [0.678914, 0.430129, 0.000000], # STO-2G coeff
    [0.444635, 0.535328, 0.154329]  # STO-3G coeff
])

# alphas2 = array([0.168855, 0.623914, 3.425251])
expon_sto = array([
    [0.270950, 0.000000, 0.000000], # STO-1G expon
    [0.151623, 0.851819, 0.000000], # STO-2G expon
    [0.109818, 0.405771, 2.227660]  # STO-3G expon
])

# here we scale the basis coeffs and exponents suitable for each Z
# we are using the STO-3G basis, which is the third row (row index = 2) so we use n to take all N columns
# remember numpy arrays can be operated on like normal numbers, i.e. w/o loops

# Atom 1 -> He
A1 = 
D1 = 

# Atom 2 -> H
A2 = 
D2 = 
# D and A are now the contraction coefficients and exponents in terms of unnormalized primitive gaussians

### One-electron integrals
Initiliaze matrices for the overlap matrix, kinetic energy matrix, and potential energy matrix. Remember they are square matrices of dimension equal to the number of electrons 

In [ ]:
Smat = zeros((nel, nel))
Tmat = zeros((nel, nel))
Vmat = zeros((nel, nel))

# Calculate the one-electron integrals: -> overlap, kinetic, and potential 
# center A is the first atom, center B the second -> origin is on A
# sum over all N-indices for each atom 
for i in range(0, N):
    for j in range(0, N):
        
        RAP = 
        # RAP2 is squared distance between center A and center P, etc. 
        
        RAP2 = 
        RBP2 = 
        Smat[0,1] += 
        Smat[1,0] = 
        
        Tmat[0,0] += 
        Tmat[0,1] += 
        Tmat[1,0] = 
        Tmat[1,1] += 
        
        # Potential from atom A + potential from atom B 
        Vmat[0,0] += 
        Vmat[0,0] += 
        
        # off-diagonal nuclear attraction to center A, etc.
        Vmat[0,1] += 
        Vmat[0,1] += 
        
        # note the symmetry, the off-diagonal elements all have the same value 
        Vmat[1,0] = 
        
        Vmat[1,1] +=
        Vmat[1,1] += 

# set diagonal of Smat to one
Smat[0,0] = 
Smat[1,1] = 

# create the core hamiltonian, H = T + V
Hmat =

In [ ]:
# let's take a look at what Smat, Hmat, Tmat Vmat are 
print_matrix(Smat,'Smat')
print_matrix(Hmat,'Hmat')
print_matrix(Tmat,'Tmat')
print_matrix(Vmat,'Vmat')

Look at that, $S_{\nu\mu}$ has off diagonal elements that are between 0 and 1!

### Two-electron integrals
Ok, we filled up the required one electron integrals, let's do the bane of every quantum chemists' existence, the two-electron repulsion integrals. Note, in general there are ${N_{\mathrm{bfn}}}^4$ two-electron integrals. In our case this would be $2^4 = 16$, however due to the inherent 8-fold symmetry, only 6 of these integrals are unique. Thus we only need to calculate those values and fill in the rest. 

Another important note, we are using chemists' notation here. Chemist' notation has an advantage of allowing the 8-fold symmetry elements to be indexed easily.  

$$ \left[ ij\mid kl\right] = \left[kl\mid ij\right] = \left[ji\mid lk\right] = \left[lk\mid ji\right] $$
$$ \left[ji\mid kl\right] = \left[lk\mid ij\right] = \left[ij\mid lk\right]  = \left[kl\mid ji\right] $$

What indices do we need to use? Using these relations we can write a nice loop that figures it out. Side note, I came up with this indexing loop and it took me a (perhaps embarrassingly) long time to figure out. I have not found a resource that discusses how to program this for symmetry, most of the time for these discussions, the elements are just given.

In [ ]:
from scipy.special import factorial, comb
def total_uni_twoeri(n):
    return (1/8)*n*(n + 1)*(n**2 + n + 2)

n = 2
rijindex = []
for i in range(n):
    for j in range(i + 1):
        for k in range(n):
            for l in range(k + 1):
                ij = int(str(i) + str(j))
                kl = int(str(k) + str(l))
                if (ij >= kl):
                    rijindex.append([i,j,k,l])
    
rijindex = array(rijindex)
print('The {:g} unique indices found are'.format(total_uni_twoeri(n)))
print(rijindex)

In [ ]:
# here we fill up the two-electron integrals
Vee = zeros((nel,nel,nel,nel))
# this is a very specific way to generate this it is not in the least sense generalized 
for i in range(0, N):
    for j in range(0, N):
        for k in range(0, N):
            for l in range(0, N):
                RAP = 
                RBP = 
                RAQ = 
                RBQ = 
                RPQ = 
                Vee[0,0,0,0] +=
                Vee[1,0,0,0] +=
                Vee[1,0,1,0] +=
                Vee[1,1,0,0] +=
                Vee[1,1,1,0] +=
                Vee[1,1,1,1] +=

# we take the least generalized way to fill the rest of the two electron integrals
# using the symmetry relationships 
Vee[0,1,0,0] = Vee[1,0,0,0] 
Vee[0,0,1,0] = Vee[1,0,0,0] 
Vee[0,0,0,1] = Vee[1,0,0,0]
Vee[0,1,1,0] = Vee[1,0,1,0]
Vee[1,0,0,1] = Vee[1,0,1,0]
Vee[0,1,0,1] = Vee[1,0,1,0]
Vee[0,0,1,1] = Vee[1,1,0,0]
Vee[1,1,0,1] = Vee[1,1,1,0]
Vee[1,0,1,1] = Vee[1,1,1,0]
Vee[0,1,1,1] = Vee[1,1,1,0]

In [ ]:
print('0 0 0 0  {: 15.10f}'.format(Vee[0,0,0,0]))
print('1 0 0 0  {: 15.10f}'.format(Vee[1,0,0,0]))
print('1 0 1 0  {: 15.10f}'.format(Vee[1,0,1,0]))
print('1 1 0 0  {: 15.10f}'.format(Vee[1,1,0,0]))
print('1 1 1 0  {: 15.10f}'.format(Vee[1,1,1,0]))
print('1 1 1 1  {: 15.10f}'.format(Vee[1,1,1,1]))

## Othogonalization
Like I said the orbitals in Hartree-Fock are not orthonormal to begin with, so we must make them orthonormal. I'll show case both schemes. 

### Canonical

We will first set up the transformation matrix in the scheme of canonical orthogonalization and create our orthonormalizer $X$. this can be acheived in two ways, the following way works for $2\times 2$ matrices
and this is how its done in Szabo & Ostlund 

In [ ]:
X  = zeros((nel, nel))
X[0,0] =  1.0/sqrt(2*(1.0 + Smat[0,1]))
X[1,0] =  X[0,0]
X[0,1] =  1.0/sqrt(2*(1.0 - Smat[0,1]))
X[1,1] = -X[0,1]

# transpose of X
XT = X.T

In [ ]:
print_matrix(X, 'X')

Let's check to see if it is orthonormal, else we're SOL

In [ ]:
check_ortho = 
print_matrix(check_ortho, "orthonormal?")

It is awesome!!!

Now, what would happen if we wanted to generalize this guy, for cases when we didn't always have only two electrons?
We can follow the matrix algebra and create a more generalized routine. For this we invoke the `eigh` function of numpy/scipy (calls `dsyev`)

In [ ]:
# u is the eigenvalues and V is the eigenvectors
u, V = eigh(Smat)
u = 1/sqrt(u)
X = V*u

XT = X.T

In [ ]:
print_matrix(X, 'X')

Hmm, why is it backwards? It has to do with the eigh function (wrapper to dysev), but I'm not totally sure why it does it that way, Inga might have a good guess, regardless it will still work, as we will see below.

In [ ]:
check_ortho = 
print_matrix(check_ortho, "orthonormal?")

### Symmetric
Ok, the second method, symmetric orthogonalization, it is constructed in a similar way to canonical orthoganlization with slight variations with respect to matrix-vector operations and has an additional step. Here it is,

In [ ]:
u, V = eigh(Smat)
# first we convert the eigenvalues from a 1D vector to a matrix 
U = sqrt(inv(u*eye(len(u))))
# from this we construct it as V * U * V^t 
A = dot(V, dot(U, V.T))

AT = A
print("Symmetric orthogonalization matrix")
print_matrix(A, 'A')
check_ortho = dot(AT, dot(Smat, A))
print_matrix(check_ortho, "orthonormal?")

In [ ]:
# we can be even more clever/lazer and use more built in routines 
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(Smat, -0.5)
# This isn't necessary, do you know why?
AT = A.T 
print_matrix(A, 'A')
check_ortho = dot(A.T, dot(Smat, A))
print_matrix(check_ortho, "orthonormal?")

Awesome, we now have two orthogonalizers, let's first start with canonical and proceed to the actual SCF cycle

## Self-Consistent Field

Let's now write our SCF iterations according to the following procedure:

#### SCF Iteration
for scf_iter in range MaxIt, do:
1. Build Fock matrix
    - Build the $\mathbf{G}$ matrix from the Coulomb and Exchange terms
    - Form the Fock matrix $\mathbf{F}$
2. RHF Energy
    - Compute electronic RHF energy
    - Compute total RHF energy 
3. Compute new orbital guess
    - Transform Fock matrix to orthonormal AO basis with $\mathbf{X}$    
    - Diagonalize $\mathbf{F}'$ for $\varepsilon$ and $\mathbf{C}'$     
    - Back transform $\mathbf{C}'$ to AO basis    
    - Form $\mathbf{P}$ from occupied orbital slice of $\mathbf{C}$ 
4. Check change in density matrix
    - Compare the change in $\mathbf{P}$ from the prior run
    - If threshold is reached exit loop
    - Else keep going 


In [ ]:
# Here we set our convergence criteria and max iterations
Crit = 1.0e-4
MaxIt = 25

# this is our initial guess of the density matrix -> a bunch of zeros
P = zeros((nel,nel))

# open the output file for writing 
f = open('HeH+_scf.out', 'w')
    
# print out the initial density matrix to the outfile 
print_matrix(P, 'P matrix', print_file=f)

# go through the iterations 
for scf_iter in range(MaxIt):
    
    # create the G matrix from the density matrix P
    # G = 2J - K
    
    G = 
    # set up fock matrix as the hamiltonian plus the G matrix
    # F = H + G
    F = ... 

    # compute the electronic energy 
    E = 
    
    # Total energy = electronic energy + nuclear repulsion energy
    ET =
    

    # Get F' = X^T * F * X
    Fprime = 
    
    # C' is the matrix of eigenvectors from the diagonalization, epsilon is the matrix of eigenvalues
    epsilon, Cprime =
    
    # Get C = X * C' 
    C = 

    # copy the density matrix from the previous iteration
    oldP = P.copy()
    
    # reset the current density matrix to zero and compute the new one
    P[:,:] = 
    for i in range(nel):
        for j in range(nel):
            P[i,j] += 

    # see how much the new density matrix has changed from the previous one
    delta = sqrt(sum((P - oldP)**2)/4)
    
    print('SCF Iteration {:2d}: Energy = {: 4.12f} dP = {: 1.5e}'.format(scf_iter + 1, ET, delta))
    
    # print stuff to file
    verbose_iter_print(f)
    
    # check the convergence criteria, if delta reaches the threshold the loop will exit, else it continues
    if delta <= Crit:
        break

# the loop has exited, determine why

# if delta is above threshold and max iterations are reached -> convergence failed
if delta > Crit and scf_iter == MaxIt:
    print_error_quit()

# if not -> convergence succeeded!! 
else:
    print_convergence_success(epsilon)
    
# close the file
f.close()

canon_energy = ET
# print the energies to the screen 
print('Electronic Energy  = {: .12f}'.format(E))
print('Total Energy       = {: .12f}'.format(ET))

Let's take a look inside the out put file. 



In [ ]:
# uncomment and run cell, there will be a lot 
results = open('HeH+_scf.out','r')
for j in results.readlines():
    print(j)
results.close()

## Properties 
Ok, now let take a look at what else we can obtain from scf output

### Charge population

$P$ is the density matrix, so it must obviously relate to the electron population via the electron density


Our final $P$ and the $S$ overlap can give us what is known as the Muliken population. We simply take the dot product of the two, though do note I formulated the equations in a slightly different way than S&O, so we need to multipy our density matrix by two to get the correct values. The reason I did this is so when we do the unrestricted hartree-fock code, it will look virtually the same, execpt have an alpha and beta part, but more on that in the next session. 

In [ ]:
mp = 
mp.diagonal()

Looking specifically at the diagonal elements of this matrix, we see the values 1.53 and 0.470 (hopefully). These are representitive of the respective electron 'charges' on the He and H atom. Note: finding the specific 'charge' on an atom is not straightforward as the electrons are not static, this gives an idea of where the electrons spend more of their time. Also, Muliken population is a very poor analysis method, and is very basis set dependent, it notable predicts positive charges for oxygen in specific situations (again see Szabo and Ostlund for a more in depth discussion) 

### Orbital occupation

Our final wavefunction is given as the $C$ matrix, and the orbital energies are defined as the eigenvalues obtained from this matrix.  

In [ ]:
print_matrix(C, 'wavefunction')

$C$ is a $2\times2$ matrix, where the first column represents the fractions of the initial wavefunctions given to form the first molecular orbital, since only one orbital is occupied, the second column represents the virtual (unoccupied) orbital. 

$$ C = \left[\begin{matrix} 
a_1 \phi_{\text{He}}^a & a_2 \phi_{\text{He}}^a \\ 
b_1 \phi_{\text{H}}^b & b_2 \phi_{\text{H}}^b \end{matrix} \right] $$

Looking at the sign we can tell the type of overlap the MO's are exhibiting

Note, in S&O, their wave function is the negative of the one I get, this is due to the orthogonalizer being backwards. If we go back and use the other way, we end up with their results, but it actually doesn't really matter, since the signs are arbitrary

$\psi_1 = -0.801924\phi_1 + -0.336792\phi_2 \rightarrow $  bonding

$\psi_2 = -0.782259\phi_1 + 1.068448\phi_2 \rightarrow $  anti-bonding


The epsilon variable is the energies of each orbital 

In [ ]:
epsilon

Of which we care only about the diagonal

In [ ]:
orbital_evals = epsilon.diagonal()
orbital_evals

We can use Koopman's theorem to determine the ionization potentials and electron affinities!

In [ ]:
IP = -orbital_evals[0]*au2eV
print('IP = {:.3f} eV'.format(IP))

In [ ]:
EA = -orbital_evals[1]*au2eV
print('EA = {:.3f} eV'.format(EA))

Ok, cool, now let's try to generalize the scf code a bit

In [ ]:
# lets put the diagonalization all into one routine 
def diag_F(F, ndocc, orthog='canonical'):
    
    if orthog == 'canonical':
        O = X
    elif orthog == 'symmetric':
        O = A

    Fp = dot(O.T, dot(F, O))
    
#   e, Cp = diag(Fp)
    # use dsyev instead of our 2x2 routine 
    e, Cp = eigh(Fp)
    
    C = dot(O, Cp)
    # occupied slice, C[:,:1] = C[:,0] (again screwy C index convention)
    C_occ = C[:,:ndocc]
    
    # einstein summation notation!
    # don't get too freaked out, this is equivalent to the previous loop we saw 
    P = einsum('ik,jk->ij', C_occ, C_occ)

    return (C, P, e)

In [ ]:
# convergence criteria and max iterations
Crit = 1.0e-4
MaxIt = 25

# how many orbitals are occupied? -> 1, because this is rhf and we have only 2 electrons
nocc = 1

# note, we start with P as all zeros, so in iteration 1, F = Hmat
# so why not just start our guess as the diag_F of Hmat
C, P, epsilon = diag_F(Hmat, nocc, orthog='symmetric')

f = open('HeH+.sym.scf.out', 'w')

print_matrix(P, 'P matrix', print_file=f)

for scf_iter in range(MaxIt):
    
    F = Hmat + formG(P)

    ET = sum(P*(Hmat + F)) + E_nuc 

    oldP = P.copy()
    
    # we'll try symmetric ortho this time
    C, P, epsilon = diag_F(F, nocc, orthog='symmetric')
    
    delta = sqrt(sum((P - oldP)**2)/4)
    
    verbose_iter_print(f)
    
    print('SCF Iteration {:2d}: Energy = {: 4.12f} dP = {: 1.5e}'.format(scf_iter + 1, ET, delta))
    
    if delta <= Crit:
        break

if delta > Crit and scf_iter == MaxIt:
    print_error_quit()

else:
    print_convergence_success(epsilon, orthog='symmetric')

f.close()

# print the energies to the screen 
symm_energy = ET
print('Electronic Energy  = {: .12f}'.format(E))
print('Total Energy       = {: .12f}'.format(ET))

We should make sure the energy values agree, we'll use the numpy function `allclose` which compares values within a given threshold

In [ ]:
allclose(canon_energy, symm_energy)

They're the same!!!!!!!!!!!!!!!!!!!!!!!!!!!

Later we will visualize this with a molden file! And if you are deathly curious, some other time I can show you how one goes about actually creating an isosurface and rendering. But I highly doubt anyone will be, no one likes the fun things.

## DIIS 
Direct inversion of the iterative subspace or the Pulay equation 

Ever wonder what that diis stuff is all about? Regardless of your answer, you're going to find out
We'll first write a function that performs the DIIS extrapolation to generate a new solution vector.

Recall the steps for the DIIS-accelerated algorithm
#### DIIS within a generic SCF Iteration
1. Compute $\mathbf{F}$, append to list of previous trial vectors
2. Compute AO orbital gradient$\mathbf{r}$, append to list of previous residual vectors
3. Compute UHF/RHF energy
3. Check convergence criteria
    - If RMSD of **r** sufficiently small, and
    - If change in SCF energy sufficiently small, break
4. Build $\mathbf{B}$ matrix from previous AO gradient vectors
5. Solve Pulay equation for coefficients $\{c_i\}$
6. Compute DIIS solution vector `F_diis` from $\{c_i\}$ and previous trial vectors
7. Compute new orbital guess with `F_diis`

In our function, we will perform steps 4-6 of the above algorithm.  What information will we need to provide our function in order to do so?  To build $\mathbf{B}$ (step 4 above) in the previous tutorial, we used:
~~~python
# Build B matrix
B_dim = len(F_list) + 1
B = empty((B_dim, B_dim))
B[-1,:] = -1
B[:, -1] = -1
B[-1, -1] = 0
for i in range(len(F_list)):
    for j in range(len(F_list)):
        B[i,j] = sum(diis_resid[i]*diis_resid[j])
~~~
We must have all previous DIIS residual vectors (`diis_resid`), as well how many previous trial vectors there are (for the dimension of $\mathbf{B}$).  To solve the Pulay equation (step 5 above):
~~~python
# Build RHS of Pulay equation 
rhs = zeros((B_dim))
rhs[-1] = -1 
      
# Solve Pulay equation for c_i's with NumPy
coeff = linalg.solve(B, rhs)
~~~
For this step, we only need the dimension of $\mathbf{B}$ (which we computed in step 4 above) and the `linalg.solve` routine from numpy/scipy, so this step doesn't require any additional arguments.  Finally, to build the DIIS Fock matrix (step 6):
~~~python
# Build DIIS Fock matrix
F_diis = zeros_like(F_list[0])
for ix in range(coeff.shape[0] - 1):
    F_diis += coeff[ix]*F_list[ix]
return F_diis
~~~
For this step, we need to know all the previous trial vectors (`F_list`) and the coefficients generated in the previous step. In the cell below, we'll write a funciton `diis_extrap()` according to the diis procedure, steps 4-6, with the above code snippets. We are essentially taking a previous list of trial vectors, `F_list` and residual vectors `diis_resid`and returning the new DIIS solution vector `F_diis`. 

In [ ]:
def diis_extrap(F_list, diis_resid):

    
    return F_diis

Now, let's throw this into our SCF routine, and see what it is cabable of. Let's also define some additional convergence criteria as well, based on the change in energy and our DIIS vector. 

In [ ]:
MaxIt = 25

# convergence of the dRMS
D_conv = 1.0e-3
# convergence of the energy 
E_conv = 1.0e-5

E_old = 0.0 
# create empty lists for the diis 
F_list = []
R_list = [] 
diis_resid = [] 

nocc = 1
C, P, epsilon = diag_F(Hmat, nocc)

for scf_iter in range(MaxIt):
    
    F = Hmat + formG(P)
    # form this intermediate M for the sake of readability
    M = 
    diis_r = 
    
    # append F and diis_r to lists
    F_list.append(F)
    R_list.append(diis_r)
    
    ET = sum(P*(Hmat + F)) + E_nuc

    C, P, epsilon = diag_F(F, nocc)
    
    if scf_iter >= 2:
        F = diis_extrap(F_list, R_list)
        
    dE = ET - E_old
    dRMS = sqrt(mean(diis_r**2))
    
    print('SCF Iteration {:2d}: Energy = {: 4.12f} dE = {: 1.5e} dRMS = {:1.5e}'.format(scf_iter+1, ET, dE, dRMS))
    
    if (abs(dE) < E_conv) and (dRMS < D_conv):
        break
    
    E_old = ET

# print the energies to the screen 
diis_energy = ET
print('\nTotal SCF Energy   = {: .12f}'.format(ET))

Check to see if this result is the same as the previous

In [ ]:
allclose(diis_energy, canon_energy)

Wow! We saved 0 steps! The savings are not yet evident, this is after all a two electron system. But trust me, and trust literature too, diis helps. 

If things failed catastrophically for you, here is the notebook [link](./aux/LetsCodeHartreeFock_InstrVer.ipynb) for the compeleted version. 

## Things to do: 
### Before next session!
1. Make a general scf function that can accept different parameters, i.e. $R$ values, $Z$ values, etc. 
1. Then make a loop that goes through a series of bond distances,
    - graph the energy curve 
    - find the minimal bond distance predicted by the STO-3G
    - look at the dissociation curve
1. Try the same for H2
  
### Anytime to gain a deeper understanding
1. Reading S&O figure out how to calculate the dipole moment!
1. Using this as a base, create a CIS code

## Next Session
We will use Psi4 to generate our integrals and investiage larger systems with our SCF routine! 
* Also to get a better grip of the einsum function, look at the following equivalent ways of constructing the G matrix 

In [ ]:
# einstein summation notation!!
def formG(P):
    g = zeros((2,2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):    
                    g[i,j] += 2*Vee[i,j,k,l]*P[k,l]
                    g[i,j] -= Vee[i,k,j,l]*P[k,l]
    return g



def formG2(P):
    J = einsum('ijkl,kl->ij', Vee, P, optimize=True)
    K = einsum('ikjl,kl->ij', Vee, P, optimize=True)
    G = 2*J - K
    return G

allclose(formG(P), formG2(P))

We can interpret einsum as for the indices `i,j,k,l` of `Vee` multiply by the indices of `k,l` of `P` and sum along the indices `i,j` from `g`

That is essentially from a four index tensor, create a two index tensor 
